In [1]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import time
import pickle
from tqdm import tqdm
print('Modules Imported')

Modules Imported


In [2]:
dt = 1./(365) #yr    
G = 39.478 #AU^3 M_sol-1 yr^-2
epsilon=0.1





In [3]:
class Body:
    
    global N,L,dt,Bodies,G, Steps
    
    def __init__(self,position,velocity,mass, name = ''):
        
        #Position n
        self.position = position
        
        #Velocity n
        self.velocity = velocity
    
        #Acceleration
        self.acceleration_i = np.zeros((3,))
        
        #Acceleration
        self.acceleration_i_1 = np.zeros((3,))
        
        #Mass
        self.mass = mass
        
        #Name 
        self.name = name
    
    def Compute_acceleration(self,Bodies):
        
        a = np.zeros((3,))
    
        for j in range(N):
        
            if Bodies[j] != self:
                
                direction = self.position - Bodies[j].position
                
                a_i = -(G*Bodies[j].mass*direction)/(np.linalg.norm(direction)**2+epsilon**2)**(3./2.)
                
                a += a_i    
            
        self.acceleration_i_1 = a 
        
        return self.acceleration_i_1
    
    def Compute_velocity(self):
        
        return self.velocity+0.5*(self.acceleration_i+self.acceleration_i_1)*dt     
    
    def Compute_position(self):
        
        return self.position+self.velocity*dt+0.5*self.acceleration_i*dt**2

    
   

In [4]:
def Angular_Momentum(pos,m):
    
    #Jz approx r * (m * v)  all these valuer are the geometric mean of my samples (sqrt(max*min))
    
    pz = 1.1*m*10.1
    
    x,y,z = pos
    
    pz = m * np.random.normal(0,0.1)
    
    px = m * np.random.normal(10,0.1)*np.cos(x)*np.sign(y)
    
    py = np.sqrt(np.abs(pz**2-px**2))*(1 + np.random.normal(0,0.03))*np.sign(-x)
    
    vx = px/m
    
    vy = py/m
    
    vz = pz/m
    
    vel = np.array([vx,vy,vz])
    
    return vel.flatten()

In [5]:
#crea las particulas

def Initialize():
 
    #este ciclo crea las posiciones y velocidades
    
    elements = [-1,1]
    probabilities = [0.5,0.5]

    for i in range(N):
            
            #We choose this sigma considering that 3 sigma will represent the 99% of our distribution.
            #And the solar system has a scale of around 30 AU
            
        x = np.random.uniform(-1.01,1.01)*np.random.choice(elements, 1, p=probabilities)[0]    
        y = np.sqrt(np.abs(1.-x**2))*(1+np.random.normal(0,0.2))*np.random.choice(elements, 1, p=probabilities)[0]    
        z = np.random.uniform(0.,0.2)
            
        pos = np.array([x,y,z])
        
        mass = np.random.normal(0.3,0.002)
    
        vel = Angular_Momentum(pos,mass)
        
        
        
        Bodies[i] = Body(pos,vel,mass,name='Particle_{}'.format(i))
    
    #este ciclo crea la aceleracion inicial para cada particula
    for i in range(N):

        Bodies[i].acceleration_i_1 = Bodies[i].Compute_acceleration(Bodies)

    print('System initialized with %d bodies'%(N))

    return Bodies

In [6]:
def particles_pos(a):
    
    positions = np.zeros((N,3))
    
    x = np.zeros((N,))
    y = np.zeros((N,))
    z = np.zeros((N,))
    
    for i in range(N):
        
        x[i] = a[i].position[0]
        y[i] = a[i].position[1]
        z[i] = a[i].position[2]    
    
    return x,y,z

In [7]:
def calc_J():
    
    J_vec_total=np.zeros((3,))
    
    for i in range(N): 
        
        J_vec_total+=Bodies[i].mass*np.cross(Bodies[i].position, Bodies[i].velocity)
        
    J_norm=np.linalg.norm(J_vec_total)
    
    return J_vec_total, J_norm

In [8]:
def Energia():
    
    val_Energia=0
    
    for i in range(N): 
        
        Energia_potencial=0
        
        for j in range(N):
        
            if Bodies[j].name!= Bodies[i].name:
                
                direction = Bodies[i].position - Bodies[j].position
                
                Energia_potencial+= -(G*Bodies[j].mass*Bodies[i].mass)/np.sqrt((np.linalg.norm(direction)**2+epsilon**2))   

        val_Energia+=0.5*Bodies[i].mass*np.linalg.norm(Bodies[i].velocity)**2+Energia_potencial
    
    return val_Energia

In [9]:
def Graficar(paso,numero_grafico):
    
    plt.clf()
    fig = plt.figure(figsize=(14,14))
    ax = fig.add_subplot(111, projection='3d')

    xs,ys,zs = particles_pos(Bodies)

    
    trayectoria_x.append(xs[0])
    trayectoria_y.append(ys[0])
    trayectoria_z.append(zs[0])

    
    
    ax.plot(trayectoria_x, trayectoria_y, trayectoria_z, label='Object 0 trayectory')
    
    ax.scatter(xs, ys, zs, c='r', marker='o')
    
    

    Lims = 2*L
        
    ax.set_xlim3d((-Lims,Lims))
    ax.set_ylim3d((-Lims,Lims))
    ax.set_zlim3d((-Lims,Lims))
    #This Sets the pannels to be White (or Transparent)
    ax.xaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
    ax.yaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
    ax.zaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
        
    # Set rotation angle to 30 degrees
    ax.view_init(azim=30)

    #ax.set_axis_off()
    ax.grid(None)
    ax.set_title('Step %d'%paso,fontsize=20)
        
    #Here we add some Info to the Image
    vector1,val2=calc_J()
    val3=Energia()
    
    textstr1 = r'Step=$%d$' % (paso)
    textstr2 = r'$\vec{J}$=$(%.2f,%.2f,%.2f)$' % (vector1[0],vector1[1],vector1[2])
    textstr3 = r'$||\vec{J}||$=$%.2f$' % (val2)
    textstr4 = r'$E_{total}$=$%.2f$' % (val3)
    
    

    # these are matplotlib.patch.Patch properties
    props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
    # place a text box in upper left in axes coords
    ax.text2D(0.05, 0.95, textstr1, transform=ax.transAxes, fontsize=18,verticalalignment='top', bbox=props)
    ax.text2D(0.05, 0.90, textstr2, transform=ax.transAxes, fontsize=18,verticalalignment='top', bbox=props)
    ax.text2D(0.05, 0.85, textstr3, transform=ax.transAxes, fontsize=18,verticalalignment='top', bbox=props)
    ax.text2D(0.05, 0.80, textstr4, transform=ax.transAxes, fontsize=18,verticalalignment='top', bbox=props)
    
    FILENAME = 'Data{}.jpg'.format(f'{numero_grafico:06}')
        
    direct = 'C:\\Users\\rmad_\\Downloads\\cojo\\outputs\\' #WIN
        
    directory = direct + FILENAME

    plt.savefig(direct + FILENAME)
            
    filenames.append(direct + FILENAME)
        
    plt.close()

In [10]:
#mueve las particulas con leapfrog

def Move():
    
    j=0
    
    for i in range(N): 
        
        Bodies[i].acceleration_i  = Bodies[i].acceleration_i_1  #solo antes de empezar a mover las particulas
                                                                #defina la acceleration_i_1 igual a la acceleration_1
    
    
    for k in tqdm(range(Steps)):
        
        time.sleep(0.01)
        
        if k%N_steps_snapshot==0:  #solo grafica la cantidad necesaria de snapshots
            
            Graficar(k,j) #graficamos el paso n+1 siempre. Incluso en el comienzo, antes de dar el primer paso el n+1 =n asi
            #que todos los pasos estan graficados
            
            j+=1

            
        
        for i in range(N):  #calcula el paso n+1 del leapfrog

            Bodies[i].acceleration_i_1  = Bodies[i].Compute_acceleration(Bodies)                 
            
        for i in range(N):  #calcula el paso n+1 del leapfrog
            
            Bodies[i].velocity = Bodies[i].Compute_velocity()
            
        for i in range(N):  #calcula el paso n+1 del leapfrog
            
            Bodies[i].position = Bodies[i].Compute_position()
        
        for i in range(N):  #pasamos las acceleration_i_1 (i+1) a acceleration_i (i) para que en el siguiente paso las recien
            #calculadas sean las viejas, y las nuevas aceleraciones sean efectivamente las nuevas en la posicion (i+1)

            Bodies[i].acceleration_i  = Bodies[i].acceleration_i_1
        
    
    #graficar el n+1 
    
    #mover el n+1 al n
        
    
        
    #salir de for de k
        
    return Bodies

In [11]:
def simulate():
    
    global N
    global Bodies
    global Steps
    global Array_Bodies
    global L
    global filenames
    global N_snapshot
    global N_steps_snapshot
       
    N=40
    Bodies = np.zeros((N,),dtype=object)
    Steps=1000
    L=2
    filenames = []
    N_snapshot=1000
    N_steps_snapshot=int(Steps/N_snapshot)
    
    global trayectoria_x
    
    global trayectoria_y
    
    global trayectoria_z
    
    trayectoria_x=[]
    
    trayectoria_y=[]
    
    trayectoria_z=[]

   
    
    
    
    Initialize()
    Move()
    
    print("The simulation has finished")


In [12]:
simulate()

System initialized with 40 bodies


100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [04:32<00:00,  3.67it/s]


The simulation has finished


<Figure size 432x288 with 0 Axes>